In [1]:
import pandas
import numpy as np
import import_ipynb
import FeatureStore

importing Jupyter notebook from FeatureStore.ipynb


In [2]:
from sklearn.preprocessing import StandardScaler

#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
def scaleDataFrame(dataframe, debugPrint = False):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe)
    dataframe = pandas.DataFrame(scaled_features, index=dataframe.index, columns=dataframe.columns)
    
    #check
    if debugPrint == True:
        for column in dataframe:
            print("[%s] mean = %.4f, variance = %.4f" % 
                  (column, dataframe[column].mean(), dataframe[column].var()))
    
    return dataframe, scaler.mean_, scaler.scale_

In [3]:
results = {}
Period = [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]
Alpha = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
Layers = [(100,100), (100,100,100)]
Activation = ['relu', 'logistic', 'tanh']
# fill results with initial zero values
for period in Period:
    results[period] = {}
    for alpha in Alpha:
        results[period][alpha] = {}
        for activation in Activation:
            results[period][alpha][activation] = {}
            for layer in Layers:
                 results[period][alpha][activation][layer] = 0.0

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from datetime import datetime


#Alpha = [0.1]; Period = [1]

sigma1 = 0
mu1 = 0
clf1 = 0
accs = {}


            
for period in Period:
    X_train = FeatureStore.getTrainDataset(period)
    y_train = FeatureStore.getTrainTarget(period)
    
    X_train, sigma, mu = scaleDataFrame(X_train, debugPrint = False)
    #print(sigma)
    #print(mu)

    for alpha in Alpha:
        for activation in Activation:
            for layer in Layers:
                print("trying Alpha = %f, period = %d, function = %s, layer config %s" 
                      % (alpha, period, activation, layer))
                kf = KFold(n_splits=3, shuffle=False, random_state=241)

                clf = MLPClassifier(hidden_layer_sizes = layer, activation=activation, solver='sgd', 
                                    alpha = alpha, max_iter = 100000, random_state=241)

                #N = 800
                #XX = X_train.iloc[:N, :]
                #yy = y_train[:N]
                XX = X_train
                yy = y_train
                #print(X_train.shape)
                #print(y_train.shape)
                #print("from", datetime.now())
                scores = cross_val_score(clf, XX, yy, scoring='accuracy', cv=kf)

                #print("spent iterations %d" % (clf.n_iter_))
                #print(clf)
                #print(vars(clf))
                #print(clf.n_iter_)
                #print(scores)
                print("accuracy=", scores.mean())
                results[period][alpha][activation] = scores.mean() 
                #print("to", datetime.now())
                #accs[period].append(scores.mean())
                if period in accs and not (accs[period] is None):
                    accs[period].append(scores.mean())
                else:
                    accs[period] = [scores.mean()]
        
for k in accs:
    accs[k] = np.max(accs[k])

trying Alpha = 0.000100, period = 1, function = relu, layer config (100, 100)
accuracy= 0.7842974718859989
trying Alpha = 0.000100, period = 1, function = relu, layer config (100, 100, 100)
accuracy= 0.7861940939136406
trying Alpha = 0.000100, period = 1, function = logistic, layer config (100, 100)
accuracy= 0.5591064683663833
trying Alpha = 0.000100, period = 1, function = logistic, layer config (100, 100, 100)
accuracy= 0.5591064683663833
trying Alpha = 0.000100, period = 1, function = tanh, layer config (100, 100)
accuracy= 0.7890269551034423
trying Alpha = 0.000100, period = 1, function = tanh, layer config (100, 100, 100)
accuracy= 0.7842947892522963
trying Alpha = 0.001000, period = 1, function = relu, layer config (100, 100)
accuracy= 0.7842974718859989
trying Alpha = 0.001000, period = 1, function = relu, layer config (100, 100, 100)
accuracy= 0.7861940939136406
trying Alpha = 0.001000, period = 1, function = logistic, layer config (100, 100)
accuracy= 0.5591064683663833
tryin

accuracy= 0.9792015409047986
trying Alpha = 1.000000, period = 2, function = logistic, layer config (100, 100)
accuracy= 0.5837410936561079
trying Alpha = 1.000000, period = 2, function = logistic, layer config (100, 100, 100)
accuracy= 0.5837410936561079
trying Alpha = 1.000000, period = 2, function = tanh, layer config (100, 100)
accuracy= 0.9782545712078289
trying Alpha = 1.000000, period = 2, function = tanh, layer config (100, 100, 100)
accuracy= 0.9810901150313333
trying Alpha = 10.000000, period = 2, function = relu, layer config (100, 100)
accuracy= 0.9536575027899391
trying Alpha = 10.000000, period = 2, function = relu, layer config (100, 100, 100)
accuracy= 0.9583869860073827
trying Alpha = 10.000000, period = 2, function = logistic, layer config (100, 100)
accuracy= 0.5837410936561079
trying Alpha = 10.000000, period = 2, function = logistic, layer config (100, 100, 100)
accuracy= 0.5837410936561079
trying Alpha = 10.000000, period = 2, function = tanh, layer config (100, 1

accuracy= 0.597932225942141
trying Alpha = 0.000100, period = 5, function = tanh, layer config (100, 100)
accuracy= 0.7899873379689244
trying Alpha = 0.000100, period = 5, function = tanh, layer config (100, 100, 100)
accuracy= 0.7880907159412825
trying Alpha = 0.001000, period = 5, function = relu, layer config (100, 100)
accuracy= 0.794711455918963
trying Alpha = 0.001000, period = 5, function = relu, layer config (100, 100, 100)
accuracy= 0.801329513262941
trying Alpha = 0.001000, period = 5, function = logistic, layer config (100, 100)
accuracy= 0.597932225942141
trying Alpha = 0.001000, period = 5, function = logistic, layer config (100, 100, 100)
accuracy= 0.597932225942141
trying Alpha = 0.001000, period = 5, function = tanh, layer config (100, 100)
accuracy= 0.7899873379689244
trying Alpha = 0.001000, period = 5, function = tanh, layer config (100, 100, 100)
accuracy= 0.7880907159412825
trying Alpha = 0.010000, period = 5, function = relu, layer config (100, 100)
accuracy= 0.79

accuracy= 0.7483421323718774
trying Alpha = 10.000000, period = 10, function = relu, layer config (100, 100)
accuracy= 0.7483474976392824
trying Alpha = 10.000000, period = 10, function = relu, layer config (100, 100, 100)
accuracy= 0.7502360717658169
trying Alpha = 10.000000, period = 10, function = logistic, layer config (100, 100)
accuracy= 0.6121126276933643
trying Alpha = 10.000000, period = 10, function = logistic, layer config (100, 100, 100)
accuracy= 0.6121126276933643
trying Alpha = 10.000000, period = 10, function = tanh, layer config (100, 100)
accuracy= 0.7492917847025495
trying Alpha = 10.000000, period = 10, function = tanh, layer config (100, 100, 100)
accuracy= 0.7492917847025495
trying Alpha = 100.000000, period = 10, function = relu, layer config (100, 100)
accuracy= 0.6121126276933643
trying Alpha = 100.000000, period = 10, function = relu, layer config (100, 100, 100)
accuracy= 0.6121126276933643
trying Alpha = 100.000000, period = 10, function = logistic, layer co

accuracy= 0.7748277749163018
trying Alpha = 0.001000, period = 40, function = relu, layer config (100, 100, 100)
accuracy= 0.7823981672246546
trying Alpha = 0.001000, period = 40, function = logistic, layer config (100, 100)
accuracy= 0.7360610352819984
trying Alpha = 0.001000, period = 40, function = logistic, layer config (100, 100, 100)
accuracy= 0.7360610352819984
trying Alpha = 0.001000, period = 40, function = tanh, layer config (100, 100)
accuracy= 0.7795599407674478
trying Alpha = 0.001000, period = 40, function = tanh, layer config (100, 100, 100)
accuracy= 0.7805122757318225
trying Alpha = 0.010000, period = 40, function = relu, layer config (100, 100)
accuracy= 0.7748277749163018
trying Alpha = 0.010000, period = 40, function = relu, layer config (100, 100, 100)
accuracy= 0.7823981672246546
trying Alpha = 0.010000, period = 40, function = logistic, layer config (100, 100)
accuracy= 0.7360610352819984
trying Alpha = 0.010000, period = 40, function = logistic, layer config (10

accuracy= 0.7729526139582797
trying Alpha = 10.000000, period = 60, function = logistic, layer config (100, 100)
accuracy= 0.7568621770109023
trying Alpha = 10.000000, period = 60, function = logistic, layer config (100, 100, 100)
accuracy= 0.7568621770109023
trying Alpha = 10.000000, period = 60, function = tanh, layer config (100, 100)
accuracy= 0.793740342518671
trying Alpha = 10.000000, period = 60, function = tanh, layer config (100, 100, 100)
accuracy= 0.78994978109709
trying Alpha = 100.000000, period = 60, function = relu, layer config (100, 100)
accuracy= 0.7568621770109023
trying Alpha = 100.000000, period = 60, function = relu, layer config (100, 100, 100)
accuracy= 0.7568621770109023
trying Alpha = 100.000000, period = 60, function = logistic, layer config (100, 100)
accuracy= 0.7568621770109023
trying Alpha = 100.000000, period = 60, function = logistic, layer config (100, 100, 100)
accuracy= 0.7568621770109023
trying Alpha = 100.000000, period = 60, function = tanh, layer

accuracy= 0.9347261567516525
trying Alpha = 0.001000, period = 250, function = logistic, layer config (100, 100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.001000, period = 250, function = tanh, layer config (100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.001000, period = 250, function = tanh, layer config (100, 100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.010000, period = 250, function = relu, layer config (100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.010000, period = 250, function = relu, layer config (100, 100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.010000, period = 250, function = logistic, layer config (100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.010000, period = 250, function = logistic, layer config (100, 100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.010000, period = 250, function = tanh, layer config (100, 100)
accuracy= 0.9347261567516525
trying Alpha = 0.010000, period = 250, function = tanh, layer confi

In [5]:
for k, v in accs.items():
    print("%d;%f" % (k, v))
    

print(clf1, sigma1, mu1)
print(results)

import json
with open('resultDNN.json', 'w') as fp:
    json.dump(results, fp)

1;0.791876
2;0.981090
3;0.895943
5;0.803223
10;0.756854
20;0.754037
40;0.783342
60;0.807950
120;0.889309
250;0.934726
MLPClassifier(activation='tanh', alpha=1000.0, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_iter=100000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=241, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False) [1.05107978e-03 9.80585321e-04 1.37135076e+08 1.37707874e+08
 2.03311258e+00 1.56745506e+01] [1.27345881e-02 1.29274745e-02 6.57771957e+07 6.61035530e+07
 1.40476317e+00 8.69552487e+00]
{1: {0.0001: {'relu': 0.7861940939136406, 'logistic': 0.5591064683663833, 'tanh': 0.7842947892522963}, 0.001: {'relu': 0.7861940939136406, 'logistic': 0.5591064683663833, 'tanh': 0.78429478

In [1]:
def backtestStrategy():
    pass